# Segmenting and Clustering Neighborhoods in Toronto

## Introduction

In this assignment, I will explore, segment, and cluster the neighborhoods in the city of Toronto. However, unlike New York, the neighborhood data is not readily available on the internet.

For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. I will scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format.

Once the data is in a structured format, I will explore and cluster the neighborhoods in the city of Toronto.

Firstly, let's download all the packages that we will require to complete the analysis
1. Folium
2. Seaborn


In [ ]:
!pip install folium

In [6]:
!pip install beautifulsoup4

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import scrapy
import requests
import json

%matplotlib inline

The neighborhood data for Toronto is available on [Wikipedia](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M)

We will scrape this page using Beautifulsoup library and transform it into a dataset

In [ ]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

